In [1]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 2 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.7
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [22]:
spark.stop()

In [2]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark import Row
import json

conf = SparkConf()

spark = (SparkSession
         .builder
         .config(conf=conf)
         .appName("test")
         .getOrCreate())

In [3]:
spark

In [4]:
! hdfs dfs -ls /labs/laba01/ml-100k

Found 23 items
-rw-r--r--   3 hdfs hdfs       6750 2022-01-06 18:46 /labs/laba01/ml-100k/README
-rw-r--r--   3 hdfs hdfs        716 2022-01-06 18:46 /labs/laba01/ml-100k/allbut.pl
-rw-r--r--   3 hdfs hdfs        643 2022-01-06 18:46 /labs/laba01/ml-100k/mku.sh
-rw-r--r--   3 hdfs hdfs    1979173 2022-01-06 18:46 /labs/laba01/ml-100k/u.data
-rw-r--r--   3 hdfs hdfs        202 2022-01-06 18:46 /labs/laba01/ml-100k/u.genre
-rw-r--r--   3 hdfs hdfs         36 2022-01-06 18:46 /labs/laba01/ml-100k/u.info
-rw-r--r--   3 hdfs hdfs     236344 2022-01-06 18:46 /labs/laba01/ml-100k/u.item
-rw-r--r--   3 hdfs hdfs        193 2022-01-06 18:46 /labs/laba01/ml-100k/u.occupation
-rw-r--r--   3 hdfs hdfs      22628 2022-01-06 18:46 /labs/laba01/ml-100k/u.user
-rw-r--r--   3 hdfs hdfs    1586544 2022-01-06 18:46 /labs/laba01/ml-100k/u1.base
-rw-r--r--   3 hdfs hdfs     392629 2022-01-06 18:46 /labs/laba01/ml-100k/u1.test
-rw-r--r--   3 hdfs hdfs    1583948 2022-01-06 18:46 /labs/laba01/ml-100k/u2.base


In [3]:
rdd = sc.textFile("/labs/laba01/ml-100k/u.data")

In [6]:
rdd.take(10)

['196\t242\t3\t881250949',
 '186\t302\t3\t891717742',
 '22\t377\t1\t878887116',
 '244\t51\t2\t880606923',
 '166\t346\t1\t886397596',
 '298\t474\t4\t884182806',
 '115\t265\t2\t881171488',
 '253\t465\t5\t891628467',
 '305\t451\t3\t886324817',
 '6\t86\t3\t883603013']

In [4]:
data_i = rdd.map(lambda x: x.split("\t")[1:3])
data_i.take(10)

[['242', '3'],
 ['302', '3'],
 ['377', '1'],
 ['51', '2'],
 ['346', '1'],
 ['474', '4'],
 ['265', '2'],
 ['465', '5'],
 ['451', '3'],
 ['86', '3']]

In [5]:
data_filtered = data_i.filter(lambda x: x[0] == '302')
data_filtered.take(10)

[['302', '3'],
 ['302', '4'],
 ['302', '4'],
 ['302', '4'],
 ['302', '3'],
 ['302', '5'],
 ['302', '3'],
 ['302', '4'],
 ['302', '5'],
 ['302', '4']]

In [46]:
data_i.filter(lambda x: x[1] == str(i+1) ).count()

6110

In [50]:
b = []
b.append('l')

In [6]:
i = 0
ans_1 = []
while i < 5:
    ans_1.append(data_filtered.filter(lambda x: x[1] == str(i+1) ).count())
    i = i + 1

In [7]:
i = 0
ans_2 = []
while i < 5:
    ans_2.append(data_i.filter(lambda x: x[1] == str(i+1) ).count())
    i = i + 1

In [8]:
ans_1

[2, 10, 46, 119, 120]

In [9]:
ans_2

[6110, 11370, 27145, 34174, 21201]

In [19]:
print(json.dumps({'hist_film': ans_1, 'hist_all': ans_2}))

{"hist_film": [2, 10, 46, 119, 120], "hist_all": [6110, 11370, 27145, 34174, 21201]}


In [21]:
with open("lab01.json", "w") as outfile:
    json.dump({'hist_film': ans_1, 'hist_all': ans_2}, outfile)